<a href="https://colab.research.google.com/github/alexfazio/crewAI-quickstart/blob/main/crewai_sequential_DirectoryReadTool_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# crewai-sequential-DirectoryReadTool-quickstart
By [Alex Fazio](https://www.x.com/alxfazio)

📂 Github repo: https://github.com/alexfazio/crewai-quickstart

This is a simplified and tested template of a **sequential** CrewAI crew.

This crew uses the `DirectoryReadTool` to provide a comprehensive **listing of directory contents**. It can recursively navigate through the specified directory, offering users a detailed enumeration of all files, including those within subdirectories.

Check the folder named `./output-files` for detailed results from each task or agent, as well as the initial output from the `DirectoryReadTool`.

Extra Requirements:
- [OpenAI](https://platform.openai.com/playground)/[Groq](https://console.groq.com/settings/organization)/[Anthropic](https://console.anthropic.com/dashboard) API Key

📚 CrewAI docs: https://docs.crewai.com/

📚 DirectoryReadTool docs: https://docs.crewai.com/tools/DirectoryReadTool/

In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
# @markdown  🔄 Restart the session and rerun the cell **if Colab requires it**.
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere

In [ ]:
# @title 🔑 Input **LLM** API Key by running this cell

import os
from getpass import getpass
# from langchain_groq import ChatGroq
# ↑ Uncomment to use Groq's API
# from langchain_anthropic import ChatAnthropic
# ↑ Uncomment to use Anthropic's API

# Set the OPENAI_API_KEY environment variable by prompting the user to enter the key
os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")
# ↑ Uncomment to use OpenAI's API
# os.environ["GROQ_API_KEY"] = getpass("Enter GROQ_API_KEY: ")
# ↑ Uncomment to use Groq's API
# os.environ["ANTHROPIC_API_KEY"] = getpass("Enter ANTHROPIC_API_KEY: ")
# ↑ Uncomment to use Anthropic's API

# Check if the 'output-files' directory exists, and create it if it doesn't
if not os.path.exists('output-files'):
    os.makedirs('output-files')

In [ ]:
# @title 🗂️ Create a Randomly Generated Sample Folder Structure

# @markdown 🔡 Variables defining the depth and the number of randomly generated folders

# @markdown If you don't have a specific preference, please **leave it as it is**.

# @markdown This is only for testing.

# @title 🗂️ Create Sample Folder Structure

import os
import random
import string

def create_nested_folders(path, depth, max_depth, num_folders, tree_prefix=""):
    if depth > max_depth:
        return ""

    tree_structure = ""

    for i in range(num_folders):
        folder_name = ''.join(random.choices(string.ascii_lowercase, k=5))
        folder_path = os.path.join(path, folder_name)
        os.makedirs(folder_path, exist_ok=True)

        # Create an empty .md file with the specified content
        md_file_path = os.path.join(folder_path, "README.md")
        with open(md_file_path, "w") as md_file:
            md_file.write("Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.")

        tree_structure += f"{tree_prefix}├── {folder_name}/\n"
        tree_structure += create_nested_folders(folder_path, depth + 1, max_depth, num_folders, tree_prefix + "│   ")

    return tree_structure

# Specify the base directory where the nested folders will be created
base_dir = "/content/nested_folders"

# Remove the base directory if it already exists
if os.path.exists(base_dir):
    import shutil
    shutil.rmtree(base_dir)

# Create the base directory
os.makedirs(base_dir)

# Set the desired depth and number of sub-folders at each level
max_depth = 2 # @param {type:"integer"}
num_folders = 2 # @param {type:"integer"}

# Create the nested folder structure and generate the visual tree
tree = create_nested_folders(base_dir, 1, max_depth, num_folders)

print("Nested folder structure created successfully.")
print("Visual tree representation:")
print(tree)
print("###")
print(f"Nested folder structure created successfully in `{base_dir}`.")

In [ ]:
# @title 🗄 Instantiate `DirectoryReadTool`

from crewai_tools import DirectoryReadTool

# Initialize the tool so the agent can read any directory's content it learns about during execution

# tool = DirectoryReadTool()

# OR

# Initialize the tool with a specific directory, so the agent can only read the content of the specified directory

directory_read_tool = DirectoryReadTool(directory="./nested_folders")

## Define Agents
In CrewAI, agents are autonomous entities designed to perform specific roles and achieve particular goals. Each agent uses a language model (LLM) and may have specialized tools to help execute tasks.

In [ ]:
# @title 🕵🏻 Define your agents

from langchain_community.chat_models import ChatOpenAI
from crewai import Agent
from textwrap import dedent

# from langchain_groq import ChatGroq
# ↑ Uncomment to use Groq's API
# from langchain_anthropic import ChatAnthropic
# ↑ Uncomment to use Anthropic's API

agent_1 = Agent(
    role=dedent((
        """
        Defines the agent's function within the crew. It determines the kind of tasks the agent is best suited for.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        Provides context to the agent's role and goal, enriching the interaction and collaboration dynamics.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        The individual objective that the agent aims to achieve. It guides the agent's decision-making process.
        """)), # This is the goal that the agent is trying to achieve
    tools=[directory_read_tool],
    allow_delegation=False,
    verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8)
    # ↑ uncomment to use OpenAI API + "gpt-4o"
    # llm=ChatGroq(temperature=0.8, model_name="mixtral-8x7b-32768"),
    # ↑ uncomment to use Groq's API + "llama3-70b-8192"
    # llm=ChatGroq(temperature=0.6, model_name="llama3-70b-8192"),
    # ↑ uncomment to use Groq's API + "mixtral-8x7b-32768"
    # llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0.8),
    # ↑ uncomment to use Anthropic's API + "claude-3-opus-20240229"
)

agent_2 = Agent(
    role=dedent((
        """
        Defines the agent's function within the crew. It determines the kind of tasks the agent is best suited for.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        Provides context to the agent's role and goal, enriching the interaction and collaboration dynamics.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        The individual objective that the agent aims to achieve. It guides the agent's decision-making process.
        """)), # This is the goal that the agent is trying to achieve
    tools=[directory_read_tool],
    allow_delegation=False,
    verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8)
    # ↑ uncomment to use OpenAI API + "gpt-4o"
    # llm=ChatGroq(temperature=0.8, model_name="mixtral-8x7b-32768"),
    # ↑ uncomment to use Groq's API + "llama3-70b-8192"
    # llm=ChatGroq(temperature=0.6, model_name="llama3-70b-8192"),
    # ↑ uncomment to use Groq's API + "mixtral-8x7b-32768"
    # llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0.8),
    # ↑ uncomment to use Anthropic's API + "claude-3-opus-20240229"
)

agent_3 = Agent(
    role=dedent((
        """
        Defines the agent's function within the crew. It determines the kind of tasks the agent is best suited for.
        """)), # Think of this as the job title
    backstory=dedent((
        """
        Provides context to the agent's role and goal, enriching the interaction and collaboration dynamics.
        """)), # This is the backstory of the agent, this helps the agent to understand the context of the task
    goal=dedent((
        """
        The individual objective that the agent aims to achieve. It guides the agent's decision-making process.
        """)), # This is the goal that the agent is trying to achieve
    tools=[directory_read_tool],
    allow_delegation=False,
    verbose=True,
    # ↑ Whether the agent execution should be in verbose mode
    max_iter=3,
    # ↑ maximum number of iterations the agent can perform before being forced to give its best answer (generate the output)
    max_rpm=100, # This is the maximum number of requests per minute that the agent can make to the language model
    llm=ChatOpenAI(model_name="gpt-4o", temperature=0.8)
    # ↑ uncomment to use OpenAI API + "gpt-4o"
    # llm=ChatGroq(temperature=0.8, model_name="mixtral-8x7b-32768"),
    # ↑ uncomment to use Groq's API + "llama3-70b-8192"
    # llm=ChatGroq(temperature=0.6, model_name="llama3-70b-8192"),
    # ↑ uncomment to use Groq's API + "mixtral-8x7b-32768"
    # llm = ChatAnthropic(model='claude-3-opus-20240229', temperature=0.8),
    # ↑ uncomment to use Anthropic's API + "claude-3-opus-20240229"
)


## Define Tasks
Tasks in CrewAI are specific assignments given to agents, detailing the actions they need to perform to achieve a particular goal. Tasks can have dependencies and context, and can be executed asynchronously to ensure an efficient workflow.

In [ ]:
# @title 📝 Define your tasks

import datetime
from crewai import Task

task_1 = Task(
    description=dedent((
        """
        A clear, concise statement of what the task entails.
        ---
        VARIABLE 1: "{var_1}"
        VARIABLE 2: "{var_2}"
        VARIABLE 3: "{var_3}"
        Add more variables if needed...
        """)),
    expected_output=dedent((
        """
        A detailed description of what the task's completion looks like.
        """)),
    agent=agent_1,
    # output_file='blog-posts/new_post.md'  # The final blog post will be saved here
    output_file=f'output-files/agent_1-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

task_2 = Task(
    description=dedent((
        """
        A clear, concise statement of what the task entails.
        ---
        VARIABLE 1: "{var_1}"
        VARIABLE 2: "{var_2}"
        VARIABLE 3: "{var_3}"
        Add more variables if needed...
        """)),
    expected_output=dedent((
        """
        A detailed description of what the task's completion looks like.
        """)),
    agent=agent_2,
    context=[task_1],
    # ↑ specify which task's output should be used as context for subsequent tasks
    output_file=f'output-files/agent_2-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

task_3 = Task(
    description=dedent((
        """
        A clear, concise statement of what the task entails.
        ---
        VARIABLE 1: "{var_1}"
        VARIABLE 2: "{var_2}"
        VARIABLE 3: "{var_3}"
        Add more variables if needed...
        """)),
    expected_output=dedent((
        """
        A detailed description of what the task's completion looks like.
        """)),
    agent=agent_3,
    context=[task_2],
    # ↑ specify which task's output should be used as context for subsequent tasks
    output_file=f'output-files/agent_3-output_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.md'
)

In [ ]:
# @title ⌨️ Define any variables you have and input them
print("## Welcome to the YOUR_CREW_NAME")
print('-------------------------------------------')
var_1 = input("What is the  to pass to your crew?\n"),
var_2 = input("What is the  to pass to your crew?\n"),
var_3 = input("What is the  to pass to your crew?\n"),
print("-------------------------------")

In [ ]:
# @title 🚀 Get your crew to work!

from crewai import Crew, Process

def main():
    # Instantiate your crew with a sequential process
    crew = Crew(
        agents=[agent_1, agent_2, agent_3],
        tasks=[task_1, task_2, task_3],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

    inputs = {
    "var_1": var_1,
    "var_2": var_2,
    "var_3": var_3
    }

    result = crew.kickoff(inputs=inputs)
    print("\n\n########################")
    print("## Here is your custom crew run result:")
    print("########################\n")
    print(result)
    
    return result

if __name__ == "__main__":
  result = main()

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))